# KAIST AI599 법률인공지능 Assignment 2
- AI Part: 60 Points
- Law Part: 40 Points
- 총점: 100 Points

이름: 강여진, 김동규, 김학성, 김형, 여인경

## AI Part
이번 과제에서는 사실관계가 주어졌을 때 적용 법률(statute)을 예측하는 모델을 만들어 봅니다. 먼저 아래와 같이 데이터를 다운로드 받습니다.

In [ ]:
!pip install -q datasets==2.4.0

     |████████████████████████████████| 365 kB 2.1 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 163 kB 65.1 MB/s 
     |████████████████████████████████| 115 kB 57.2 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 


In [ ]:
import datasets

dataset_card = "lbox/lbox_open"
task = "statute_classification"
data = datasets.load_dataset(dataset_card, task)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating test2 split: 0 examples [00:00, ? examples/s]

Dataset lbox_open downloaded and prepared to /root/.cache/huggingface/datasets/lbox___lbox_open/statute_classification/0.2.0/2f5d77c154367b6d59788348092531a670a563485f5ff8ab8e5d00704718f4bf. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

데이터를 개별 확인해 보면, 아래와 같은 형태를 띄고 있는 것을 볼 수 있습니다.

In [ ]:
data_v = data["validation"]
data_v[40]

{'id': 1286,
 'casetype': 'criminal',
 'casename': '교통사고처리특례법위반(치사)',
 'statutes': ['교통사고 처리특례법 제3조 제1항', '형법 제268조'],
 'facts': '피고인은 (차량번호 1 생략) 케이(K)5 택시의 운전업무에 종사하는 사람이다.\n피고인은 2021. 5. 19. 03:40경 위 택시를 운전하여 춘천시 B 앞 편도 2차로 도로를 퇴계사거리 방면에서 석사사거리 방면으로 시속 약 48킬로미터로 진행하던 중 전방 및 좌우 주시의무를 게을리 하고 조향 및 제동 장치를 제때 조작하지 못한 업무상 과실로 마침 전방 횡단보도 신호가 적색임에도 피고인 진행방향 좌측에서 우측으로 위 도로를 무단횡단 하던 피해자 C(남, 72세)을 미처 발견하지 못하고 피고인 택시의 앞 범퍼 부분으로 피해자를 들이받아 그 충격으로 피해자로 하여금 2021. 5. 21. 21:40경 춘천시 D 소재 E병원에서 다발성장기부전 등으로 사망에 이르게 하였다.'}

여기서 저희는 사실관계(facts)를 입력값으로 하여 적용법률(statutes)를 예측하고자 합니다. 적용법률이 여러개일 수 있기 때문에, `list` 형태인 점을 주의하세요.

**Problem AI.1** *(10 points)*. `statutes`는 text string으로 돼 있습니다. 분류문제로 치환하기 위해서는, 먼저 train 시에 관측되는 법률의 가지수를 찾으세요 *(5 points)*. 또한 validation에서 관측되는 볍률중 학습시에 본적이 없는 법률이 있는데요, (법률 단위가 아닌) 예제단위로 이 비율을 계산하세요 *(5 points)*. 

In [ ]:
data_t = data['train']
statutes = sorted(set.union(*map(set, data_t['statutes'])))
#print(statutes)
print(len(statutes))

151


In [ ]:
cnt = 0
for d in data_v:
    cnt += not set(d['statutes']).issubset(statutes)
print(cnt / len(data_v))

0.018115942028985508


**Problem AI.2** *(30 points)*. Hugging Face의 `multilingual-bert-cased`을 활용해 사실관계->법률분류 모델을 학습하고 validation 데이터에 대해 정확성을 보고하세요. 여러개의 적용 법률중 하나라도 맞추면 정답으로 간주합니다. 학습 때 못 본 법률을 고려해, "Unknown"에 해당하는 분류를 추가하고 학습 때 고려할 수 있도록 하세요 (다만, 학습 때 못 본 법률의 경우 "Unknown"으로 매핑되면 오답으로 간주). Colab 기준 학습 30분 이내에 70% 이상의 정확성이 나올 수 있도록 합니다. 수업 중 공개한 예제코드를 활용하셔도 좋습니다. 

In [ ]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6

     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
     |████████████████████████████████| 1.3 MB 2.1 MB/s 
     |████████████████████████████████| 707 kB 2.1 MB/s 
     |████████████████████████████████| 5.9 MB 64.1 MB/s 
     |████████████████████████████████| 419 kB 59.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.1 which is incompatible.


In [ ]:
from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, statutes, tokenizer, data, batch_size=16, max_input_len=512):
      super().__init__()
      self.statutes = statutes
      self.tokenizer = tokenizer
      self.data = data
      self.batch_size = batch_size
      self.max_input_len = max_input_len 
    
    def setup(self, stage):
        pass
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self._collate_fn)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['validation'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def _collate_fn(self, batch):
        inputs = self.tokenizer([x['facts'] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x['statutes'] for x in batch]

        return inputs, labels

In [ ]:
class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class'):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
    
    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 
    
    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 

    def test_epoch_end(self, outputs):
        self._test_epoch_end(outputs)
   
    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > 0.5)[0]] for logit in logits]

        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs):
        avg_loss = np.mean([x['loss'] for x in outputs])
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)
        
        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')

        # target_ids = random.sample(range(len(gts)), 5)
        # for target_id in target_ids:
        #     print(f"GT: {gts[target_id]}\t\t\tPR: {prs[target_id]}")

        for id in range(len(gts)):
            if prs[id] not in gts[id]:
                print(f"[{id}] GT: {gts[id]}\t\t\tPR: {prs[id]}")
                print(f"\tdetail facts: [{data['validation'][id]}]")

    def _test_epoch_end(self, outputs):
        avg_loss = np.mean([x['loss'] for x in outputs])
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)
        
        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')

        for id in range(len(gts)):
            if prs[id] not in gts[id]:
                print(f"[{id}] GT: {gts[id]}\t\t\tPR: {prs[id]}")
                print(f"\tdetail facts: [{data['test'][id]}]")

    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate,}]
        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)

        return {"optimizer": optimizer}

In [ ]:
backbone_card = 'bert-base-multilingual-cased'

# data
statutes = ['Unknown'] + sorted(set.union(*map(set, data['train']['statutes'])))
tokenizer = AutoTokenizer.from_pretrained(backbone_card)
data_module = StatutesDataModule(statutes, tokenizer, data, batch_size=24, max_input_len=256)

# model
backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(statutes))
model = StatutesClassifier(statutes, backbone, learning_rate=1e-5, mode='multi-class')

# trainer
n_gpus = 1#torch.cuda.device_count()
trainer = pl.Trainer(max_epochs=10, gpus=n_gpus, fast_dev_run=not True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                          | Params
------------------------------------------------------------
0 | backbone  | BertForSequenceClassification | 177 M 
1 | criterion | CrossEntropyLoss              | 0     
------------------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.881   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

avg_loss: 4.984483003616333
ACC: 0.0
[0] GT: ['형법 제298조']			PR: 공직선거법 제85조 제3항
	detail facts: [{'id': 180, 'casetype': 'criminal', 'casename': '강제추행', 'statutes': ['형법 제298조'], 'facts': '피고인은 2020. 1. 22. 02:00경 서울 서초구 B 소재 **** 주점 2번 테이블에서 술을 마시다가 화장실을 다녀오던 중, 갑자기 테이블 바깥쪽에 앉아있던 피해자 C(여, 39세)의 왼쪽 가슴을 3회 만져 피해자를 강제로 추행하였다.'}]
[1] GT: ['형법 제298조']			PR: 공직선거법 제85조 제3항
	detail facts: [{'id': 181, 'casetype': 'criminal', 'casename': '강제추행', 'statutes': ['형법 제298조'], 'facts': '1. 2020. 7. 27.자 범행\n피고인은 2020. 7. 27. 18:00경 영주시 B에 있는 피해자 C(여, 76세)의 주거지에 찾아와 "같이 살자"라고 말을 하며 피해자의 상의를 걷어 올려 양손으로 피해자의 양쪽 가슴을 수차례 주무른 후 피해자의 손을 잡아채 피고인의 성기에 가져가 피고인의 성기를 만지게 하고, 이어 피해자의 바지 위에 손을 얹어 피해자의 음부를 만졌다.\n2. 2020. 8. 8.자 범행\n피고인은 2020. 8. 8. 09:00경 제1항의 피해자 주거지에서, 마당에 있다가 집 안으로 들어가는 피해자를 따라 들어가 피해자의 상의 옷 안으로 한 손을 넣어 가슴을 주무르고, 피해자의 바지 속에 손을 넣어 피해자의 음부를 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다.'}]
[2] GT: ['형법 제298조']			PR: 공직선거법 제256조 제3항 제2호
	detail facts: [{'id': 182, 'casetype': 'criminal', 'casename': '강제추행', 'statu

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

Validation: 0it [00:00, ?it/s]

avg_loss: 1.2720602949460347
ACC: 0.8840579710144928
[30] GT: ['공직선거법 제103조 제3항', '공직선거법 제256조 제3항 제1호']			PR: 공직선거법 제257조 제1항 제1호
	detail facts: [{'id': 1180, 'casetype': 'criminal', 'casename': '공직선거법위반', 'statutes': ['공직선거법 제103조 제3항', '공직선거법 제256조 제3항 제1호'], 'facts': '피고인은 B, C 지역 D구의회 의원으로, 2020. 4. 15. 제21대 국회의원선거 서울 E 지역구에 F정당 후보로 출마하여 당선된 G의 선거사무원이었던 사람이다.\n누구든지 선거기간 중 선거에 영향을 미치게 하기 위하여 향우회ㆍ종친회ㆍ동창회ㆍ단합대회 또는 야유회, 그 밖의 집회나 모임을 개최할 수 없다.\n피고인은 2020. 4. 초경 위 G 후보자를 위하여 선거구민인 B 소재 재건축 추진 아파트단지 주민들을 대상으로 간담회 모임을 개최하기로 마음먹고, 2020. 4. 9. 14:00경부터 서울 H 1층 ‘I\' 카페에 J 등 선거구민 약 19명을 모이게 하고 이들에게 미리 G 후보자를 소개하였다.\nG은 2020. 4. 9. 14:42경 위 장소에 도착하여 "저는 기왕이면 B에서 표를 많이 주셨으면 좋겠어요.", "남아있는 기간이 얼마 안 되기 때문에 간곡히 좀 부탁드립니다. 화끈하게 좀 도와주십시오. G이가 일할 수 있게 도와주시면..."이라고 말하는 등 선거운동을 하였고, 이어서 피고인도 "가장 적임자는 저는 G 후보님이라고 아 여러분 앞에서 진짜 확신 드립니다.", "G 의원님 아 용기 우리 주시고요, 우리 B에서 좀 팍팍 밀어줍시다. 여러분 원하신다면 박수 부탁드립니다."라며 G 후보에 대한 지지발언을 하는 등 선거운동을 하였다.\n이로써 피고인은 선거기간 중에 선거에 영향을 미치게 하기 위하여 모임을 개최하였다. 증거의 요지\n1. 피고인의 법정진술\n1. 피고인에

Validation: 0it [00:00, ?it/s]

avg_loss: 1.2123785714308422
ACC: 0.8840579710144928
[30] GT: ['공직선거법 제103조 제3항', '공직선거법 제256조 제3항 제1호']			PR: 공직선거법 제257조 제1항 제1호
	detail facts: [{'id': 1180, 'casetype': 'criminal', 'casename': '공직선거법위반', 'statutes': ['공직선거법 제103조 제3항', '공직선거법 제256조 제3항 제1호'], 'facts': '피고인은 B, C 지역 D구의회 의원으로, 2020. 4. 15. 제21대 국회의원선거 서울 E 지역구에 F정당 후보로 출마하여 당선된 G의 선거사무원이었던 사람이다.\n누구든지 선거기간 중 선거에 영향을 미치게 하기 위하여 향우회ㆍ종친회ㆍ동창회ㆍ단합대회 또는 야유회, 그 밖의 집회나 모임을 개최할 수 없다.\n피고인은 2020. 4. 초경 위 G 후보자를 위하여 선거구민인 B 소재 재건축 추진 아파트단지 주민들을 대상으로 간담회 모임을 개최하기로 마음먹고, 2020. 4. 9. 14:00경부터 서울 H 1층 ‘I\' 카페에 J 등 선거구민 약 19명을 모이게 하고 이들에게 미리 G 후보자를 소개하였다.\nG은 2020. 4. 9. 14:42경 위 장소에 도착하여 "저는 기왕이면 B에서 표를 많이 주셨으면 좋겠어요.", "남아있는 기간이 얼마 안 되기 때문에 간곡히 좀 부탁드립니다. 화끈하게 좀 도와주십시오. G이가 일할 수 있게 도와주시면..."이라고 말하는 등 선거운동을 하였고, 이어서 피고인도 "가장 적임자는 저는 G 후보님이라고 아 여러분 앞에서 진짜 확신 드립니다.", "G 의원님 아 용기 우리 주시고요, 우리 B에서 좀 팍팍 밀어줍시다. 여러분 원하신다면 박수 부탁드립니다."라며 G 후보에 대한 지지발언을 하는 등 선거운동을 하였다.\n이로써 피고인은 선거기간 중에 선거에 영향을 미치게 하기 위하여 모임을 개최하였다. 증거의 요지\n1. 피고인의 법정진술\n1. 피고인에

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
trainer.test(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

avg_loss: 1.3301461338996887
ACC: 0.8768115942028986
[23] GT: ['형법 제136조 제1항', '형법 제314조']			PR: 형법 제314조 제1항
	detail facts: [{'id': 795, 'casetype': 'criminal', 'casename': '공무집행방해, 업무방해', 'statutes': ['형법 제136조 제1항', '형법 제314조'], 'facts': '1. 업무방해\n피고인은 2021. 3. 17. 02:40경 원주시 B에 있는 피해자 C(여, 48세)이 운영하는 ‘D식당\'에서 그곳에 있던 손님과 시비가 되어 젓가락과 휴대전화로 테이블을 수회 내리치고, 큰 소리로 욕설을 하는 등 소란을 피우고, 그곳 종업원에 의하여 위 식당 밖으로 퇴거당한 후에도 손으로 식당 창문을 수회 두드리는 등의 방법으로 약 20분간 소란을 피웠다.\n이로써 피고인은 위력으로써 피해자의 식당영업 업무를 방해하였다.\n2. 공무집행방해\n피고인은 2021. 3. 17. 03:04경 전항 기재 장소에서 "여자 손님이 혼자 난동을 피워요"라는 112신고를 받고 출동한 원주경찰서 E지구대 소속 경장 F(40세)으로부터 신분증 제시를 요구받자, "신분증 여기 있다"라고 소리를 치며 신분증을 들고 있던 손으로 F의 얼굴 부위를 1회 때렸다.\n이로써 피고인은 경찰공무원의 112신고 사건처리에 관한 정당한 직무집행을 방해하였다.'}]
[24] GT: ['형법 제245조']			PR: 형법 제283조 제1항
	detail facts: [{'id': 990, 'casetype': 'criminal', 'casename': '공연음란', 'statutes': ['형법 제245조'], 'facts': '1. 2020. 9. 18.경 범행\n피고인은 2020. 9. 18. 13:40경 서울 중랑구 B에 있는 주택가 골목에서 그곳을 지나던 피해자 C(여, 27세)를 향해 바지와 팬티를 내려 피고인의 성기를 꺼내어 보이면서 "빨아줘

[{}]

**Problem AI.3** *(20 points)*. 2번 문제가 일반 분류 문제였다면, multi-label classification으로 접근해서 학습하고 정확성을 보고하세요. 각 예제마다 모든 적용 법률을 맞췄을 때만 정답으로 간주합니다. 

In [ ]:
backbone_card = 'bert-base-multilingual-cased'

# data
statutes = ['Unknown'] + sorted(set.union(*map(set, data['train']['statutes'])))
tokenizer = AutoTokenizer.from_pretrained(backbone_card)
data_module = StatutesDataModule(statutes, tokenizer, data, batch_size=24, max_input_len=256)

# model
backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(statutes))
model = StatutesClassifier(statutes, backbone, learning_rate=2e-5, mode='multi-label')

# trainer
n_gpus = 1#torch.cuda.device_count()
trainer = pl.Trainer(max_epochs=50, gpus=n_gpus, fast_dev_run=not True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                          | Params
------------------------------------------------------------
0 | backbone  | BertForSequenceClassification | 177 M 
1 | criterion | BCEWithLogitsLoss             | 0     
------------------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.881   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


avg_loss: 0.6865037381649017
ACC: 0.0
GT: ['도로교통법 제148조의2 제1항', '도로교통법 제152조 제1항', '도로교통법 제43조', '도로교통법 제44조 제1항']			PR: ['Unknown' '공직선거법 제106조 제1항' '공직선거법 제113조 제1항' '공직선거법 제115조'
 '공직선거법 제167조 제3항' '공직선거법 제237조 제1항 제2호' '공직선거법 제240조 제1항'
 '공직선거법 제241조 제1항' '공직선거법 제252조 제1항' '공직선거법 제255조 제1항 제11호'
 '공직선거법 제255조 제1항 제17호' '공직선거법 제255조 제1항 제2호' '공직선거법 제255조 제1항 제9호'
 '공직선거법 제255조 제2항 제4호' '공직선거법 제256조 제3항 제2호' '공직선거법 제257조 제1항 제1호'
 '공직선거법 제57조의3 제1항' '공직선거법 제64조 제1항' '공직선거법 제87조 제1항 제3호'
 '공직선거법 제90조 제1항 제1호' '공직선거법 제95조 제1항' '교통사고 처리특례법 제3조 제2항 제11호'
 '교통사고 처리특례법 제3조 제2항 제2호' '교통사고 처리특례법 제3조 제2항 제9호' '도로교통법 제148조의2'
 '도로교통법 제148조의2 제1항' '도로교통법 제148조의2 제2항' '도로교통법 제148조의2 제3항 제1호'
 '도로교통법 제148조의2 제3항 제2호' '도로교통법 제148조의2 제3항 제3호' '도로교통법 제152조 제1항'
 '도로교통법 제44조' '도로교통법 제44조 제1항' '병역법 제69조 제1항' '병역법 제86조' '병역법 제87조 제3항'
 '병역법 제88조 제1항' '병역법 제88조 제1항 제1호' '병역법 제88조 제1항 제2호' '병역법 제88조 제1항 제3호'
 '성폭력범죄의 처벌 등에 관한 법률 제14조 제1항' '성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항'
 '아동복지법 제17조 제7호' '아동복지법 제1항 제2호' '

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

Validation: 0it [00:00, ?it/s]

avg_loss: 0.010885826700056592
ACC: 0.782608695652174
GT: ['형법 제156조']			PR: []
GT: ['형법 제245조']			PR: ['형법 제245조']
GT: ['도로교통법 제148조의2 제1항', '도로교통법 제44조 제1항', '도로교통법 제44조 제2항']			PR: ['도로교통법 제148조의2 제1항' '도로교통법 제44조 제1항' '도로교통법 제44조 제2항']
GT: ['형법 제260조 제1항', '형법 제261조']			PR: ['형법 제260조 제1항' '형법 제261조']
GT: ['형법 제231조', '형법 제234조']			PR: ['형법 제231조' '형법 제234조']


Validation: 0it [00:00, ?it/s]

avg_loss: 0.011697154569750031
ACC: 0.7862318840579711
GT: ['전자금융거래법 제49조 제4항 제2호', '전자금융거래법 제6조 제3항 제2호']			PR: ['전자금융거래법 제49조 제4항 제2호' '전자금융거래법 제6조 제3항 제2호']
GT: ['형법 제347조 제1항']			PR: ['형법 제347조 제1항']
GT: ['정보통신망 이용 촉진 및 정보보호 등에 관한 법률 제70조 제2항']			PR: []
GT: ['교통사고 처리특례법 제3조 제1항', '교통사고 처리특례법 제3조 제2항 제1호', '교통사고 처리특례법 제3조 제2항 제6호', '형법 제268조']			PR: ['교통사고 처리특례법 제3조 제1항' '교통사고 처리특례법 제3조 제2항 제1호' '형법 제268조']
GT: ['형법 제319조 제1항']			PR: ['형법 제319조 제1항']


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


## Law Part
위 모델을 활용하여 사실관계->법률 예측 모델의 분석 및 기획을 진행합니다.

In [2]:
# 이미지 첨부를 위한 gdrive 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Problem Law.1** *(20 points)*. AI.2 모델의 출력값과 정답이 다른 경우는 주로 어떤 패턴이 있나요? 2가지의 패턴을 찾고 이유를 분석하세요. 

1. **범죄 사실(facts)이 긴 경우**
    1. 오답 빈도수가 가장 높은 죄명은 협박죄, 허위사실 적시 사이버 명예훼손죄, 허위사실 적시 명예훼손죄 등임. 해당 죄들은 구체적인 발언 내용, 행위 내용을 범죄사실에 명시해야 하기 때문에 범죄사실이 비교적 길어짐.
        
        ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
        
    2. 모델은 facts에서 주목해야 하는 주요 키워드에 적절한 attention을 주지 않고 facts 전체를 동일한 weight로 학습함. 이 경우 facts 내 전반적인 context에만 집중해 답을 내게 되어, 오답 발생함. 주요 키워드로는 다음과 같은 내용이 있음.
        1. **사이버 상에서 발생했는지? (ex. ‘카톡방’, ‘메시지’, ‘인터넷 커뮤니티’…)** → 이 경우 형법이 아니라 정통망법이 적용됨
        2. **적시한 내용이 ‘사실’인지 ‘허위사실’인지?** → 사실적시 명예훼손죄와 허위사실적시 명예훼손죄를 가르게 됨
        3. **발언 내용이 ‘모욕’인지 협박인지? (ex. 이로써 피고인은 ~ 모욕하였다.)** → 모욕죄와 명예훼손죄를 가르게 됨
        4. **발언 내용이 ‘성’과 관련된 내용인지?** → 모욕, 협박죄와 성폭법 위반 통매음을 가르게 됨
        등이 있음.
    3. 예를 들어, 모델이 **정통망법 위반 허위사실적시 사이버 명예훼손죄를 협박죄로 잘못 판단**한 경우,
        1. GT: ['정보통신망 이용 촉진 및 정보보호 등에 관한 법률 제70조 제2항'] 
        PR: 형법 제283조 제1항
        [{'id': 8292, 'casetype': 'criminal', 'casename': '정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)', 'statutes': ['정보통신망 이용 촉진 및 정보보호 등에 관한 법률 제70조 제2항'], 'facts': '피고인과 피해자 B, 피해자 C은 D 주식회사에서 근무하는 버스기사로 직장 동료이다.\n1. 2020. 11. 5.자 범행\n피고인은 2020. 11. 5. 21:19 무렵 알 수 없는 곳에서 휴대전화를 이용해 D 주식회사 E번 버스를 운행하는 기사 49명이 참여하는 카카오톡 단체 채팅방에서「B 씨한테 전화하면 고소당하는 건가요?? 그렇다면! 무슨 이유인가요?? B 씨한테 전화해서 고소당하신 분 계신가요?? 그러다보니 이상한 생각이 들더라고요?? 둘의 관계가 뭐지?? 그냥 동료라면 C이가 나한테 절대 할 수도 없는... 해서는 안 되는 말인데....!?!?!? 그렇다고 피를 나눈 남매도 아니고...... 도대체 왜? C이가 저런 말을 했을까?? 머리가 깨질 것 같네요~!!! 이것도 저것도 아니면....??? 혹시???? 그렇고 그런 사이????? 에이~!!!! 설마~!!!!!! 설마가 사람 잡는다고 하긴 하던데.... 아이고!! 머리가 더 깨질 것 같습니다~!!!!」라고 말하였다.\n이로써 피고인은 피해자들을 비방할 목적으로 정보통신망을 통하여 피해자들이 불륜관계에 있다는 취지의 허위사실을 공공연하게 게시하여 피해자들의 명예를 훼손하였다.\n2. 2020. 11. 26.자 범행\n피고인은 2020. 11. 26. 14:05 무렵 알 수 없는 곳에서 휴대전화를 이용해 D 주식회사 E번 버스를 운행하는 기사 49명이 참여하는 카카오톡 단체 채팅방에서「그리고 둘이 그렇고 그런 사이.... 이 말도 성희롱인가요? 없는 말한 것도 아닌데 어느 누가 너! 누구한테 전화하지 마라 고소당하기 싫으면... 이런 소리 들으면 고민 안 하겠습니까?? 고소당하게 생겼는데. 별의별 생각이 다 들죠~?!?!."라는 메시지를 게시하였다.\n이로써 피고인은 피해자들을 비방할 목적으로 정보통신망을 통하여 피해자들이 불륜관계에 있다는 취지의 허위사실을 공공연하게 게시하여 피해자들의 명예를 훼손하였다.'}]
        2. ”이로써 피고인은~”에 해당하는 문장, “허위사실” 등이 허위사실 적시 사이버 명예훼손죄의 주요 키워드이지만, “어느 누가 너! 누구한테 전화하지 마라 고소당하기 싫으면…” 같은 전반적인 context를 보고 협박죄로 판단한 것으로 보임.
2. **‘죄수 관계’, ‘형법-특별법, 행정법 간의 관계’ 등 법률 각 조항 간 관계성을 누락한 경우**
    1. 죄수 - 공범 규정
        1. 피고인이 공동정범, 방조범에 해당하는 경우에는 형법 제30조 등 공범에 관한 규정이 적용되어야 함.
            
            ![picture](https://drive.google.com/uc?id=1utfn7Bhdxck1IqR0xXtlieMz9id80jga)
            
        2. 하지만 모델은 공범에 관한 법률 내용을 모름. 따라서 범죄사실에 “피고인은 ~~와 공모하여”라는 ‘공범성’을 명시하는 표현이 들어가더라도 공범 규정을 답하지 않는 오류가 발생함.
        3. 과실범, 부작위범, 미수 등에 대해서도 같은 문제가 발생할 것으로 보임.
    2. 죄수 - 상상적 경합
        1. 상상적 경합이란, 한 가지 행위가 수죄에 해당하는 경우를 말함.
        예를 들어 공무집행방해죄와 업무방해죄가 이에 해당함. 판례에 따르면 이는 한 가지 행위가 수 죄에 해당하는 ‘상상적 경합’ 관계임. 
            
            ![picture](https://drive.google.com/uc?id=1ulQFZylhp5uK4p9Z0aiLe4pgYtjC8jGh)
            
        2. 모델은 수죄 사이의 상상적 경합, 실체적 경합 관계도 모름. 따라서 “공무집행 방해죄, 업무방해죄”가 아니라 “업무방해죄” 한 가지 답만을 내놓았음.
    3. 형법-특별법 간 관계성
        1. 다음과 같이 형법에 우선하여 특별법이 적용되어야 하는 경우가 있음.
            1. 특가법(특정범죄 가중처벌 등에 관한 법률)
            1억원 이상의 뇌물죄 가중처벌, 상습 강도 등 가중처벌, 도주치사상의 뺑소니 가중처벌, 위험운전 치사상 등
            2. 폭처법(폭력행위 등 처벌에 관한 법률)
            집단 폭행, 집단 협박, 집단 재물손괴, 집단 상해 등 집단, 상습 폭력범 처벌 등
            3. 성폭법(성폭력범죄의 처벌 등에 관한 특례법)
            특수강도강간(주거침입+강간), 합동강간 등
            4. 아청법(아동, 청소년의 성 보호에 관한 법률)
            아동을 대상으로 하는 강간, 강제추행 등
            5. 교특법(교통사고 처리 등에 관한 특례법)
            6. 정통망법(정보통신망 이용촉진 및 정보 보호 등에 관한 법률)
            명예훼손이 사이버 상에서 이뤄진 경우
        2. 모델은 형법-특별법 간의 관계성을 모르기 때문에 다음과 같은 오류가 발생했음.
            
            ![picture](https://drive.google.com/uc?id=149MJcnoXVVOM-UdeTLpIVk1CmAeAJAPd)
            
            1. “운전업무에 종사하는 사람” 즉 택시 운전사가 교통사고를 낸 경우, 형법 제268조 업무상 과실치사상죄 뿐만 아니라 교특법 제3조 제1항이 함께 적용되어야 하나, 형법 제268조만 답함.
            2. 버스 등 운송수단의 운전자를 폭행한 경우 형법 제260조 제1항의 단순 폭행죄가 아니라 특가법 제5조의10 제1항이 적용되어야 하나, 형법 제260조 제1항만 답함.
    4. 행정법의 의무규정-벌칙규정 관계
        1. 아동복지법, 공직선거법, 전자금융거래법, 병역법, 도로교통법 등은 행정법임. 행정법은 수범자의 의무를 규정하는 “의무 규정”과 이를 위반했을 때의 처벌 내용을 규정하는 “벌칙 규정”으로 구성되어 있음.
        2. 하지만 모델은 각 행정법의 의무규정, 벌칙규정 쌍(pairs)의 관계를 모르기 때문에 “벌칙 규정”만 명시하는 일이 발생하는 것으로 보임.
            
            ![picture](https://drive.google.com/uc?id=1yI1-Lj3-DaCsgmXYh0fGErPjmPtBwKFO)
            
            ![picture](https://drive.google.com/uc?id=1NVJolPuD0yRrePuFLPLNES5gdOonujVv)
            
            1. 예를 들어 모델이 답한대로 “전자금융거래법 제49조 제4항 제2호” 위반의 건이라고 가정하면, 판사는 이에 대한 의무 규정인 “동법 제6조 제3항 제2호 또는 제3호”를 함께 명시했을 것임. 그러나 모델은 오직 벌칙 규정인 제49조 제4항 제2호만 명시했음.
            2. 또한 아동복지법 제71조 제1항 제2호 위반 시에도, 의무규정인 아동복지법 제17조 제3호를 명시했어야 함.
            
            ![picture](https://drive.google.com/uc?id=1Yq3nAUMGcRXbHEOOgbiMJCPulRUU7D9t)
            
        3. 나아가 train set이 형법, 각 특별법, 각 행정법 간 골고루 분포하고 있지 않기 때문에, 특히나 각 행정법은 training이 부족한 문제점도 가지고 있음. 
        train/valid/test 간의 데이터 분포 차이는 크지 않지만,
            
            ![picture](https://drive.google.com/uc?id=1YS3n-wcVpFdhftIFVgQRv2sIo9UhPpBi)
            
            1. Unknown 법률의 존재
            2. class imbalance로 인해 representation layer 자체가 제대로 학습되지 않았을 가능성이 큼


**Problem Law.2** *(10 points)*. 현재 모델은 법률의 내용을 모른체로 분류를 진행합니다. 모델이 법률의 내용을 알게 된다면 어떤 이점이 있을까요? 현재는 오답을 내고 있는 예제 중 법률 내용을 모델이 고려할 수 있다면 정답을 맞출 수도 있는 경우가 있나요?


1. 전술한 사안 중 2번, 즉 **법률 각 조항 간 관계성**을 고려한 경우 정답률이 상승할 것이라고 생각함.
2. 이를 위해서는 다음과 같은 내용을 train시켜야 함.
    1. “피고인은 ~~와 공모하여”와 같이 공범성을 표현하는 문장이 있으면 형법 제30조~제34조의 공범 규정을 답하게 한다.
    2. 부작위범, 미수범 등에 대해서도 형법 제25조, 형법 제18조, 형법 제342조 등 각 죄의 미수범 처벌규정을 답하게 한다.
    3. 상상적 경합 관계의 죄 조합을 학습시킨다. 
        1. 무면허운전-음주운전
        2. 교통사고 시 차량 손괴-인적 상해
        3. 공무집행방해-준강도
        4. 사기-알선수뢰죄
        5. 강간, 강도 수단으로-감금죄
        등
    4. 특별법에 해당하는 사안은 특별법을 우선으로 답하게 한다.
        1. 1억 원 이상의 뇌물 관련 범죄인 경우
        2. 공동으로 집단 폭력행위가 있었던 경우
        3. 합동강간, 주거침입 이후 강간 등
        4. 아동을 대상으로 하는 범죄의 경우(이때 아동의 연령 고려)
        등
    5. 행정법은 벌칙규정-의무규정 쌍을 학습시킨다.
        1. 도로교통법 제148조의2 제1항 - 도로교통법 제44조 제1항 or 제2항
        2. 도로교통법 제148조의2 제3항 - 도로교통법 제44조 제3항
        등
3. + 범죄 사실 중 결론 문장에 더 큰 weight 부여
    1. 범죄사실의 내용이 긴 경우에는 모델에게 주목해야 하는 문장의 weight를 높여서 학습시킨다. 
    2. 범죄사실이 길어지면 마지막 결론 문장에 “결국 피고인은 ~하였다.” “이로써 피고인은 ~하였다.”와 같이 범죄의 구성요건을 요약하여 정리하는 경우가 많은데, 해당 문장의 weight를 높이면 정답률이 상승할 것이라고 생각한다.

![picture](https://drive.google.com/uc?id=1l-OxeIKIyTX8LBbopi1Ip3mhX3n5A6Cz)

![picture](https://drive.google.com/uc?id=1GnvBDg4OtPoeiatX7wloi-dWHeOgVxgs)

![picture](https://drive.google.com/uc?id=1gLfoX33KkbyGbJ8zNcWc6Itp9kUle9HF)


**Problem Law.3** *(10 points)*. 위 모델을 활용하여 (추가학습 없이) 유사 사실관계 검색을 하려 합니다. 어떻게 접근 할 수 있을지 간단하게 설명해 주세요.


- 모델에 사실관계를 넣으면 feature vector (e.g., 모델의 representation layer에서 추출한 encoding, 모델 마지막 레이어에서 softmax 함수를 적용하기 전의 logit) 를 얻을 수 있음
- 두 feature 간의 유사도 (e.g. cosine-similarity)  계산
- 두 사실관계로부터 모델을 통해 추출한 feature가 높은 유사도 값을 가진다면 두 사실관계가 유사할 가능성이 높음
- 검색하려는 사실관계를 입력받아 모델로 feature를 추출한 후, feature space 내에서 유사도가 높은 다른 사실관계들을 찾아내서 보여주면 됨

## *첨부: 정답이 잘못된 사안*

1. 특가법 5조의1조는 존재하지 않음
엘박스 확인결과 특가법 5조의10이 잘못 입력된 것으로 보임
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2010.png)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2011.png)
    
2. 형법 97조는 항, 호가 존재하지 않는 단일 조문임. 이 내용은 엘박스에도 잘못 등록되어 있는 것 같음.
전기통신 사업법의 몇조 몇호 위반인지 명시되어 있지 않고 형법과 섞여서 써 있음.
[https://lbox.kr/case/서울북부지방법원/2021고정475](https://lbox.kr/case/%EC%84%9C%EC%9A%B8%EB%B6%81%EB%B6%80%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2021%EA%B3%A0%EC%A0%95475)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2012.png)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2013.png)
    
3. 교통사고 처리 특례법 제3조 제1항과 형법 268조인데, 교특법 268조라고 잘못 기재되어 있음
[https://lbox.kr/case/창원지방법원진주지원/2021고단1317](https://lbox.kr/case/%EC%B0%BD%EC%9B%90%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90%EC%A7%84%EC%A3%BC%EC%A7%80%EC%9B%90/2021%EA%B3%A0%EB%8B%A81317)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2014.png)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2015.png)
    
4. 하나의 판결문에 2가지 범죄사실이 기재되어 있는 데이터인 경우
원칙적으로 본 모델은 1가지 범죄사실만을 찾도록 설계되어 있을 것이므로, 잘못된 출제된 문제라고 생각함
또한 판결문에서 “제314조 제1항”으로 기재될 내용이 “제314조”로만 기재되어 있음
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2016.png)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2017.png)
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2018.png)
    
5. 오류는 아니지만 마찬가지로 2개 범죄사실이 한 판결문에 동시에 들어있는 경우
사문서 위조죄-위조사문서 행사죄
    
    ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
    ![Untitled](%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%A9%E1%86%BC%20%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%202fa5d6d8c1f1401fb252f2e2f7121a25/Untitled%2019.png)